In [5]:
import json
import math
import scipy
import ast
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statistics 
import seaborn as sns
from numpy.random import randn
from statistics import mode
%matplotlib inline

In [9]:
%%bash
./thesisDataExport.sh

console log


2021-05-12T08:41:45.046+0200	could not connect to server: server selection error: server selection timeout, current topology: { Type: ReplicaSetNoPrimary, Servers: [{ Addr: pearup-shard-00-01.r81v8.mongodb.net:27017, Type: Unknown, State: Connected, Average RTT: 0, Last error: connection() : dial tcp 18.157.202.220:27017: i/o timeout }, { Addr: pearup-shard-00-02.r81v8.mongodb.net:27017, Type: Unknown, State: Connected, Average RTT: 0, Last error: connection() : dial tcp 3.65.178.249:27017: i/o timeout }, { Addr: pearup-shard-00-00.r81v8.mongodb.net:27017, Type: Unknown, State: Connected, Average RTT: 0, Last error: connection() : dial tcp 3.64.42.179:27017: i/o timeout }, ] }
2021-05-12T08:42:15.187+0200	could not connect to server: server selection error: server selection timeout, current topology: { Type: ReplicaSetNoPrimary, Servers: [{ Addr: pearup-shard-00-02.r81v8.mongodb.net:27017, Type: Unknown, State: Connected, Average RTT: 0, Last error: connection() : dial tcp 3.65.178.249

In [8]:
!./thesisDataExport.sh

2021-05-12T08:39:14.381+0200	could not connect to server: server selection error: server selection timeout, current topology: { Type: ReplicaSetNoPrimary, Servers: [{ Addr: pearup-shard-00-01.r81v8.mongodb.net:27017, Type: Unknown, State: Connected, Average RTT: 0, Last error: connection() : dial tcp 18.157.202.220:27017: i/o timeout }, { Addr: pearup-shard-00-02.r81v8.mongodb.net:27017, Type: Unknown, State: Connected, Average RTT: 0, Last error: connection() : dial tcp 3.65.178.249:27017: i/o timeout }, { Addr: pearup-shard-00-00.r81v8.mongodb.net:27017, Type: Unknown, State: Connected, Average RTT: 0, Last error: connection() : dial tcp 3.64.42.179:27017: i/o timeout }, ] }
2021-05-12T08:39:44.471+0200	could not connect to server: server selection error: server selection timeout, current topology: { Type: ReplicaSetNoPrimary, Servers: [{ Addr: pearup-shard-00-02.r81v8.mongodb.net:27017, Type: Unknown, State: Connected, Average RTT: 0, Last error: connection() : dial tcp 3.65.178.249

In [4]:
pupils = json.load(open('pupils.json', 'r'))    # result will be a python dictionary
classrooms = json.load(open('classrooms.json', 'r'))

print(len(pupils))
print(len(classrooms))

808
231


# Klassen wählen, die das KI Paket gemacht haben

In [3]:
# taskseriesIDs vom KI Paket:
KI_taskseries_ids = ["AI-1-v2", "AI-2-v2", "AI-3-v2", "AI-4-v2", "ML-1", "ML-2", "ML-3", "Classification-1", "Clustering-1", "Classification-2", "Clustering-2", "LimitsML", "EvalML", "Impact-1", "Impact-2", "Impact-3", "Impact-4", "AIClosing"]

classIDsKI = []
pupilsKI = []

for pupilnumber in range(len(pupils)): #für jeden schüler
    for tasknumber in range(len(pupils[pupilnumber]["solvedTaskSeries"])): #für jede seiner tasks
        if("taskSeriesId" in pupils[pupilnumber]["solvedTaskSeries"][tasknumber].keys()):
            if(pupils[pupilnumber]["solvedTaskSeries"][tasknumber]["taskSeriesId"] in KI_taskseries_ids):
                for sectionnumber in range(len(pupils[pupilnumber]["solvedTaskSeries"][tasknumber]["currentSections"])): #für diese tasks, alle sections
                    for subsectionnumber in range(len(pupils[pupilnumber]["solvedTaskSeries"][tasknumber]["currentSections"][sectionnumber]["currentSubSections"])):
                        if("exercise" in pupils[pupilnumber]["solvedTaskSeries"][tasknumber]["currentSections"][sectionnumber]["currentSubSections"][subsectionnumber].keys()):
                            if pupils[pupilnumber]["classId"] not in classIDsKI:
                                classIDsKI.append(pupils[pupilnumber]["classId"])
                            if pupils[pupilnumber] not in pupilsKI:
                                pupilsKI.append(pupils[pupilnumber])

KI_classes = {classID: {"count": 0} for classID in classIDsKI}

for pupil in pupilsKI:
    KI_classes[pupil["classId"]]["count"] += 1

print(KI_classes)
print("")

# Jetzt nur die Klassen mit genügend SuS auswählen
final_pupils_KI = []
min_class_size = 10

for pupil in pupilsKI:
    if(KI_classes[pupil["classId"]]["count"] >= min_class_size):
        final_pupils_KI.append(pupil)

print("Zur Zeit gibt es " + str(len(final_pupils_KI)) + " sus, die das KI Paket bearbeitet haben")


{'uxfuiLT4xZA6yvE6L': {'count': 2}, 'qFEyvypMn4sAwLwtc': {'count': 26}, 'XdLWZx5FrxRsoKap8': {'count': 2}, 'y7j9eKDPsqNkAycuy': {'count': 1}, '6Z2MmkKQKebb7nz8P': {'count': 3}, '4fyXxCKaLbCBaPbYE': {'count': 1}, 'xYWgeSPkADwSmhdmm': {'count': 2}, 'GkQSJvPTRWsLnpyR9': {'count': 14}, '7cWBWtLEcLSYDygDc': {'count': 1}, 'GbPhM46pBxWirBfin': {'count': 1}, 'kCmX662oevP4hJpSZ': {'count': 1}, 'dJR3L7tcMwMRSydWE': {'count': 1}, 'TeQTDWWjCiMxJgNfg': {'count': 6}, 'hZ9Hi9DLZSm5JMbNm': {'count': 10}, 'Zx6YpkyjLyHd8Rmvp': {'count': 4}, 'NCm8NNbAmNsur5S33': {'count': 19}, 'Xr2bFE3qSrJk5EsNH': {'count': 1}, 'xskRD6rct9ASrjR9u': {'count': 1}, 'L4KttRnMpBPMidCfQ': {'count': 1}, 'E6MAFLQAPpbaq56Lq': {'count': 2}, 'xXcRs55AwSdHJwNqx': {'count': 2}, '7XiJ56zCm4omf95iZ': {'count': 2}, 'Xr7LXctGQp5cWjsRx': {'count': 3}}

Zur Zeit gibt es 69 sus, die das KI Paket bearbeitet haben


## DF mit classId und pupilId konstruieren

In [4]:
class_list = [{"pupilId": pupil["_id"], "classId": pupil["classId"]} for pupil in final_pupils_KI]
class_df = pd.DataFrame(class_list)

class_df.head()

,pupilId,classId
0,bniRycMi5bwrXjvGy,qFEyvypMn4sAwLwtc
1,sqBXKssFvS8pFYAMv,qFEyvypMn4sAwLwtc
2,Kd6FuRsrq6QvPrkKp,qFEyvypMn4sAwLwtc
3,6bBwnX2mwAp2NAwNR,qFEyvypMn4sAwLwtc
4,ZTvPk2i77LDC8pFmu,qFEyvypMn4sAwLwtc


# Chat df

In [5]:
date = []
sentBy = []
messages = []
request = []

row_nr = 0
for pupil in final_pupils_KI:
    for message in pupil["chat"]["messages"]:
        date.append(message["date"])
        sentBy.append(message["sentBy"])
        messages.append(message["message"])
        if "request" in message.keys():
            request.append(message["request"])
        else:
            request.append(False)

#zipped_list = list(zip((date, sentBy, messages, request)))
#chat_df = pd.DataFrame(zipped_list, columns =["date", "sentBy", "message", "request"])

chat_df = pd.DataFrame(date)
chat_df["sentBy"] = sentBy
chat_df["message"] = messages
chat_df["request"] = request

chat_df.head()
#print(zipped_list)

,0,sentBy,message,request
0,2021-02-23T07:33:51+00:00,sWm4d454H7CgZk7xP,Hier könnt ihr euch individuell aber auch mir ...,False
1,2021-02-23T07:37:04+00:00,auto,feedback,"{'taskSeriesId': 'AI-1-v2', 'userId': 'YQmE5bR..."
2,2021-02-23T08:05:17+00:00,auto,feedback,"{'taskSeriesId': 'AI-4-v2', 'userId': 'YQmE5bR..."
3,2021-02-23T08:10:27+00:00,auto,feedback,"{'taskSeriesId': 'ML-1', 'userId': 'YQmE5bRt4Q..."
4,2021-02-23T08:12:44+00:00,auto,feedback,"{'taskSeriesId': 'ML-2', 'userId': 'YQmE5bRt4Q..."


# DF mit allen bearbeitete Aufgaben im KI Paket --> pupil_df

In [6]:
tasktypes = ["blockly", "multipleChoice", "freetext", "survey", "inputField", "surveySlider", "noTask"]
exercise_keys = ['state', 'submitted', 'subType', 'evaluation', 'falseAttempts', 'hintsTaken', 'rating', 'teacherFeedback', "history"]

columns = ["pupilId", "taskId", "taskSeriesId", "duration"] + exercise_keys
pupil_df = pd.DataFrame(columns=columns)

row_nr = 0
for pupil in final_pupils_KI:
    for task in pupil["solvedTaskSeries"]:
        sec_nr = 1
        for section in task["currentSections"]:
            subsec_nr = 1
            for subsection in section["currentSubSections"]:
                # eine neue Zeile hinzufügen für jede bearbeitete Aufgabe und id columns füllen
                #df = df.append({"pupilId": pupil["_id"], "taskId": str(task["taskSeriesId"])+"-"+str(sec_nr)+"-"+str(subsec_nr),'taskSeriesId': task["taskSeriesId"], "sec_nr": sec_nr, "subsec_nr": subsec_nr}, ignore_index=True)
                pupil_df = pupil_df.append({"pupilId": pupil["_id"], "taskId": str(task["taskSeriesId"])+"-"+str(sec_nr)+"-"+str(subsec_nr),'taskSeriesId': task["taskSeriesId"], "duration": subsection["duration"]}, ignore_index=True)
                subsec_nr += 1
                if("exercise" in subsection.keys()):
                    # hier wird der eigentliche state der aufgabe in die reihe eingetragen
                    for key in subsection["exercise"].keys():
                        pupil_df.at[row_nr, key] = subsection["exercise"][key]
                row_nr += 1
            sec_nr +=1
    
# aufgaben loswerden, die nicht submitted wurden!
pupil_df.dropna(subset=['submitted'], inplace=True)

#die antworten der einen falschen aufgabe berichtigen
'''
for index in pupil_df.index:
    if(pupil_df.loc[index, "taskId"] == "Classification-2-1-5"):
        if ("2" in str(pupil_df.loc[index, "falseAttempts"])): #die SuS, die es eigentlich richtig hatten
            pupil_df.loc[index, "falseAttempts"] = np.nan
            pupil_df.loc[index, "evaluation"] = "correct"
        elif(pupil_df.loc[index, "pupilId"]=="gooproPtWeTH6dtcB"): #der eine schüler, der es falsch hatte
            pupil_df.loc[index, "falseAttempts"] = [str(pupil_df.loc[index, "state"])]
            pupil_df.loc[index, "evaluation"] = "false"
'''
# die eine aufgabe mit der falschen lösung droppen
index_names = pupil_df[ pupil_df['taskId'] == "Classification-2-1-5" ].index 
pupil_df.drop(index_names, inplace = True) # drop the respective rows

pupil_df.head()


,pupilId,taskId,taskSeriesId,duration,state,submitted,subType,evaluation,falseAttempts,hintsTaken,rating,teacherFeedback,history,revision
0,bniRycMi5bwrXjvGy,AI-1-v2-1-1,AI-1-v2,40,92,True,surveySlider,NaN,NaN,NaN,NaN,NaN,[],NaN
1,bniRycMi5bwrXjvGy,AI-1-v2-1-2,AI-1-v2,119,"<p>Künstliche Intelligenz ist ein Algorithmus,...",True,freetext,correct,NaN,NaN,"{'content': 3, 'format': 5, 'creativity': 4}","Inhaltlich gut, etwas genauer antworten. Muste...",[],NaN
2,bniRycMi5bwrXjvGy,AI-1-v2-2-1,AI-1-v2,268,"[0, 0, 1, 0, 0]",True,survey,NaN,NaN,NaN,NaN,NaN,[],NaN
4,bniRycMi5bwrXjvGy,AI-2-v2-1-1,AI-2-v2,94,"[True, True, False, True]",True,multipleChoice,correct,NaN,NaN,NaN,NaN,[],NaN
6,bniRycMi5bwrXjvGy,AI-3-v2-1-1,AI-3-v2,43,"[False, False, False]",True,multipleChoice,correct,NaN,NaN,NaN,NaN,[],NaN


## DF mit allen Tasks

In [7]:
#taskSeries Laden
taskSeries = json.load(open('taskSeries.json', 'r'))    # result will be a python dictionary

In [8]:
interaction_subtypes = ['multipleChoice', 'blockly', 'surveySlider', 'survey', 'inputField', 'freetext']
columns = ["taskId", "subType", "question", "answers", "minLabel", "maxLabel", "hints", "sampleSolution"]
task_df = pd.DataFrame(columns=columns)

row_nr = 0
for taskSerie in taskSeries:
    sec_nr = 1
    if taskSerie["_id"] in KI_taskseries_ids:
        for section in taskSerie["sections"]:
            subsec_nr = 1
            for subsection in section["subSections"]:
                for elem in subsection["elements"]:
                    if(elem["subType"] in interaction_subtypes):
                        task_df = task_df.append({"taskId": str(taskSerie["_id"])+"-"+str(sec_nr)+"-"+str(subsec_nr), "subType": elem["subType"], "question": elem["question"]["de"]}, ignore_index=True)
                        #task_df = task_df.append({"taskId": str(taskSerie["_id"])+"-"+str(sec_nr)+"-"+str(subsec_nr),'taskSeriesId': taskSerie["_id"], "sec_nr": sec_nr, "subsec_nr": subsec_nr, "subType": elem["subType"], "question": elem["question"]["de"]}, ignore_index=True)
                        if(elem["subType"] == "surveySlider"):
                            task_df.at[row_nr, "minLabel"] = elem["specificData"]["minLabel"]["de"]
                            task_df.at[row_nr, "maxLabel"] = elem["specificData"]["maxLabel"]["de"]
                        elif(elem["subType"] == "freetext"):
                            task_df.at[row_nr, "sampleSolution"] = elem["sampleSolution"]["de"]
                        elif(elem["subType"] == "survey"):
                            task_df.at[row_nr, "answers"] = elem["specificData"]["answers"]
                        elif(elem["subType"] == "multipleChoice"):
                            task_df.at[row_nr, "answers"] = elem["specificData"]["answers"]
                            task_df.at[row_nr, "sampleSolution"] = elem["sampleSolution"]["universal"]
                            if(len(elem["hints"]) > 0):
                                #task_df.at[row_nr, "hints"] = [elem["hints"] for i in range(len(elem["hints"]))]
                                task_df.at[row_nr, "hints"] = elem["hints"]#[0]["de"]
                        elif(elem["subType"] == "inputField"):
                            task_df.at[row_nr, "sampleSolution"] = elem["sampleSolution"]["universal"]
                            if(len(elem["hints"]) > 0):
                                #task_df.at[row_nr, "hints"] = [elem["hints"] for i in range(len(elem["hints"]))]
                                task_df.at[row_nr, "hints"] = elem["hints"]#[0]["de"]
                        row_nr += 1
                subsec_nr +=1
            sec_nr +=1
            
task_df.head()

,taskId,subType,question,answers,minLabel,maxLabel,hints,sampleSolution
0,AI-1-v2-1-1,surveySlider,"Wie gut weißt du schon, was KI ist?",NaN,Gar nicht,Sehr gut,NaN,NaN
1,AI-1-v2-1-2,freetext,Was ist deiner Meinung nach Künstliche Intelli...,NaN,NaN,NaN,NaN,Die Schülerinnen und Schüler (SuS) sollen besc...
2,AI-1-v2-2-1,survey,Wie viele deiner Zeichnungen hat die KI erkann...,"[{'de': '0', 'en-US': '0'}, {'de': '1-2', 'en-...",NaN,NaN,NaN,NaN
3,AI-2-v2-1-1,multipleChoice,"Kannst du erkennen, wo KI überall drin steckt?...",[{'de': 'Der Newsfeed eines Social Network nut...,NaN,NaN,"[{'de': 'Denk an die Beispiele, die wir uns am...","[True, True, False, True]"
4,AI-3-v2-1-1,multipleChoice,Was für Sachen können KI-Anwendungen?,"[{'de': 'Die Anwendung, die den Facebook-Feed ...",NaN,NaN,[{'de': 'Kann überhaupt eine dieser Anwendunge...,"[False, False, False]"


## DFs mergen

In [9]:
new_df = pd.merge(pupil_df.drop(['subType'], axis = 1), task_df, on='taskId')
new_df.drop(columns=['history', "submitted"], inplace=True)

In [10]:
df = pd.merge(class_df, new_df, on="pupilId")
df.head()

,pupilId,classId,taskId,taskSeriesId,duration,state,evaluation,falseAttempts,hintsTaken,rating,teacherFeedback,revision,subType,question,answers,minLabel,maxLabel,hints,sampleSolution
0,bniRycMi5bwrXjvGy,qFEyvypMn4sAwLwtc,AI-1-v2-1-1,AI-1-v2,40,92,NaN,NaN,NaN,NaN,NaN,NaN,surveySlider,"Wie gut weißt du schon, was KI ist?",NaN,Gar nicht,Sehr gut,NaN,NaN
1,bniRycMi5bwrXjvGy,qFEyvypMn4sAwLwtc,AI-1-v2-1-2,AI-1-v2,119,"<p>Künstliche Intelligenz ist ein Algorithmus,...",correct,NaN,NaN,"{'content': 3, 'format': 5, 'creativity': 4}","Inhaltlich gut, etwas genauer antworten. Muste...",NaN,freetext,Was ist deiner Meinung nach Künstliche Intelli...,NaN,NaN,NaN,NaN,Die Schülerinnen und Schüler (SuS) sollen besc...
2,bniRycMi5bwrXjvGy,qFEyvypMn4sAwLwtc,AI-1-v2-2-1,AI-1-v2,268,"[0, 0, 1, 0, 0]",NaN,NaN,NaN,NaN,NaN,NaN,survey,Wie viele deiner Zeichnungen hat die KI erkann...,"[{'de': '0', 'en-US': '0'}, {'de': '1-2', 'en-...",NaN,NaN,NaN,NaN
3,bniRycMi5bwrXjvGy,qFEyvypMn4sAwLwtc,AI-2-v2-1-1,AI-2-v2,94,"[True, True, False, True]",correct,NaN,NaN,NaN,NaN,NaN,multipleChoice,"Kannst du erkennen, wo KI überall drin steckt?...",[{'de': 'Der Newsfeed eines Social Network nut...,NaN,NaN,"[{'de': 'Denk an die Beispiele, die wir uns am...","[True, True, False, True]"
4,bniRycMi5bwrXjvGy,qFEyvypMn4sAwLwtc,AI-3-v2-1-1,AI-3-v2,43,"[False, False, False]",correct,NaN,NaN,NaN,NaN,NaN,multipleChoice,Was für Sachen können KI-Anwendungen?,"[{'de': 'Die Anwendung, die den Facebook-Feed ...",NaN,NaN,[{'de': 'Kann überhaupt eine dieser Anwendunge...,"[False, False, False]"


# DF bearbeiten

In [11]:
mask_freetext = df['subType'].str.startswith('freetext')
df.loc[mask_freetext, 'state'] = df.loc[mask_freetext, 'state'].str.replace("<p>","")
df.loc[mask_freetext, 'state'] = df.loc[mask_freetext, 'state'].str.replace("</p>","")

In [12]:
answer_length_array = []
numberFalseAttempts = []
numberHints=[]

for i, row in df.iterrows():
    if row["subType"] == "freetext":
        answer_length_array.append(int(len(row["state"].split())))
        #answer_length_array.append(len(string.split()) for string in row["state"])
    else:
        answer_length_array.append(np.nan)
    #if row["subType"] == "inputField":
    if row["falseAttempts"] != row["falseAttempts"]:
        numberFalseAttempts.append(0)
    else:
        numberFalseAttempts.append(len(row["falseAttempts"]))
    if row["hints"] != row["hints"]:
        numberHints.append(np.nan)
    else:
        numberHints.append(len(row["hints"]))
    #else:
    #    numberFalseAttempts.append(np.nan)
#print(answer_length_array)

In [13]:
df["number words in answer"] = answer_length_array
df["numberFalseAttempts"] = numberFalseAttempts
df["numberHints"] = numberHints

In [15]:
rating_content = []
rating_format = []
rating_creativity = []

for i, task in df.iterrows():
    if((task["subType"] == "freetext") & (task["evaluation"] != "pending")):
        rating_content.append(task["rating"]["content"])
        rating_format.append(task["rating"]["format"])
        rating_creativity.append(task["rating"]["creativity"])
    else:
        rating_content.append(np.nan)
        rating_format.append(np.nan)
        rating_creativity.append(np.nan)
        
        

df["rating_content"] = rating_content    
df["rating_format"] = rating_format 
df["rating_creativity"] = rating_creativity

df.head()

,pupilId,classId,taskId,taskSeriesId,duration,state,evaluation,falseAttempts,hintsTaken,rating,...,minLabel,maxLabel,hints,sampleSolution,number words in answer,numberFalseAttempts,numberHints,rating_content,rating_format,rating_creativity
0,bniRycMi5bwrXjvGy,qFEyvypMn4sAwLwtc,AI-1-v2-1-1,AI-1-v2,40,92,NaN,NaN,NaN,NaN,...,Gar nicht,Sehr gut,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
1,bniRycMi5bwrXjvGy,qFEyvypMn4sAwLwtc,AI-1-v2-1-2,AI-1-v2,119,"Künstliche Intelligenz ist ein Algorithmus, de...",correct,NaN,NaN,"{'content': 3, 'format': 5, 'creativity': 4}",...,NaN,NaN,NaN,Die Schülerinnen und Schüler (SuS) sollen besc...,18.0,0,NaN,3.0,5.0,4.0
2,bniRycMi5bwrXjvGy,qFEyvypMn4sAwLwtc,AI-1-v2-2-1,AI-1-v2,268,"[0, 0, 1, 0, 0]",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
3,bniRycMi5bwrXjvGy,qFEyvypMn4sAwLwtc,AI-2-v2-1-1,AI-2-v2,94,"[True, True, False, True]",correct,NaN,NaN,NaN,...,NaN,NaN,"[{'de': 'Denk an die Beispiele, die wir uns am...","[True, True, False, True]",NaN,0,1.0,NaN,NaN,NaN
4,bniRycMi5bwrXjvGy,qFEyvypMn4sAwLwtc,AI-3-v2-1-1,AI-3-v2,43,"[False, False, False]",correct,NaN,NaN,NaN,...,NaN,NaN,[{'de': 'Kann überhaupt eine dieser Anwendunge...,"[False, False, False]",NaN,0,1.0,NaN,NaN,NaN


## DataFrame splitten nach tasktypes

In [16]:
surveySlider = df[df["subType"]=="surveySlider"].copy()
surveySlider.drop(["hints", "hintsTaken", "numberFalseAttempts", "sampleSolution", "answers", "falseAttempts", "rating", "teacherFeedback", "number words in answer", "rating_content", "rating_format", "rating_creativity"], axis = 1, inplace=True) 


In [17]:
inputFields = df[df["subType"]=="inputField"].copy()
inputFields.drop(["minLabel", "maxLabel", "answers", "rating", "teacherFeedback", "number words in answer", "rating_content", "rating_format", "rating_creativity"], axis = 1, inplace=True) 


In [18]:
multipleChoice = df[df["subType"]=="multipleChoice"].copy()
multipleChoice.drop(["minLabel", "maxLabel","rating", "teacherFeedback", "number words in answer", "rating_content", "rating_format", "rating_creativity"], axis = 1, inplace=True)


In [19]:
freetext = df[df["subType"] == "freetext"].copy()
freetext.drop(["minLabel", "maxLabel", "answers", "hints", "hintsTaken", "numberFalseAttempts", "rating"], axis = 1, inplace=True) 


## False attempts bei Freitexts cleanen

In [20]:
# getting rid of p tags in false attempts
for entry in freetext[np.where(freetext["falseAttempts"] == freetext["falseAttempts"], True, False)]["falseAttempts"]:
    entry[0] = entry[0].replace("<p>", "").replace("</p>", "")

## Pupil performance df

In [22]:
nr_tasks_per_pupil = []
total_duration_per_pupil = []

for pupilId in df["pupilId"].unique():
    nr_tasks_per_pupil.append(len(df[df["pupilId"]==pupilId]))
    total_duration_per_pupil.append(sum(df[df["pupilId"]==pupilId]["duration"])/60)

pupil_performance = {"pupilId": df["pupilId"].unique(), "nr_tasks_per_pupil": nr_tasks_per_pupil, "total_duration in minutes": total_duration_per_pupil}
pupil_performance_df = pd.DataFrame(pupil_performance)
pupil_performance_df["average duration per task in minutes"] = pupil_performance_df["total_duration in minutes"]/pupil_performance_df["nr_tasks_per_pupil"]

pupil_performance_df = pupil_performance_df.set_index('pupilId')

pupil_performance_df.head()

,nr_tasks_per_pupil,total_duration in minutes,average duration per task in minutes
pupilId,,,
bniRycMi5bwrXjvGy,66,70.233333,1.064141
sqBXKssFvS8pFYAMv,66,66.650000,1.009848
Kd6FuRsrq6QvPrkKp,6,37.383333,6.230556
6bBwnX2mwAp2NAwNR,35,114.550000,3.272857
ZTvPk2i77LDC8pFmu,66,59.400000,0.900000


In [23]:
task_df.set_index('taskId', inplace=True)

In [24]:
taskIds = ["AI-1-v2-1-1", "AI-4-v2-1-4", "EvalML-1-1", "ML-2-1-1", 'EvalML-1-5', "AI-4-v2-1-5", 'EvalML-1-2', 'EvalML-1-6', 'AIClosing-1-1', 'AIClosing-1-3', 'AIClosing-1-4', "AI-1-v2-1-2", 'AI-4-v2-1-3', 'ML-2-1-2', 'EvalML-1-7', 'AIClosing-1-2', 'AIClosing-1-5', 'LimitsML-1-2', "AI-2-v2-1-1", 'EvalML-1-4', 'AI-4-v2-1-1', 'AI-4-v2-1-2', 'AI-3-v2-1-2', 'EvalML-1-3', 'Classification-1-1-1', "Clustering-1-1-1", 'ML-3-1-2', 'ML-3-1-3','ML-3-1-4','EvalML-1-9', 'Classification-2-1-8', "Impact-1-1-1", 'EvalML-1-8', 'LimitsML-1-1', 'AIClosing-1-6','AIClosing-1-7','AIClosing-1-8']

for taskId in taskIds:
    pupil_performance_df[taskId] = np.nan
    if(task_df.loc[taskId, "subType"] == "freetext"):
        for ratingType in ["rating_content", "rating_format", "rating_creativity"]:
            pupil_performance_df[taskId+"_"+ratingType] = np.nan
        for pointType in ["_correct_points", "_incorrect_points", "_irrelevant_points"]:
            pupil_performance_df[taskId+pointType] = np.nan

In [25]:
# je nach aufgabentyp die richtigen felder erstellen und befüllen
for pupilId in pupil_performance_df.index:
    for taskId in taskIds:
        if(taskId in df[df["pupilId"] == pupilId]["taskId"].unique()):
            if (df[(df["pupilId"] == pupilId) & (df["taskId"] == taskId)][["subType"]].values == "surveySlider"):
                pupil_performance_df.loc[pupilId, taskId] = df[(df["pupilId"] == pupilId)  & (df["taskId"] == taskId)][["state"]].values[0][0]
            elif((df[(df["pupilId"] == pupilId)  & (df["taskId"] == taskId)][["subType"]].values == "freetext") & (df[(df["pupilId"] == pupilId)  & (df["taskId"] == taskId)][["evaluation"]].values == "correct")):
                pupil_performance_df.loc[pupilId, taskId] = df[(df["pupilId"] == pupilId)  & (df["taskId"] == taskId)][["state"]].values[0][0]
                pupil_performance_df.loc[pupilId, taskId+"_rating_content"] = df[(df["pupilId"] == pupilId)  & (df["taskId"] == taskId)][["rating_content"]].values[0][0]
                pupil_performance_df.loc[pupilId, taskId+"_rating_format"] = df[(df["pupilId"] == pupilId)  & (df["taskId"] == taskId)][["rating_format"]].values[0][0]
                pupil_performance_df.loc[pupilId, taskId+"_rating_creativity"] = df[(df["pupilId"] == pupilId)  & (df["taskId"] == taskId)][["rating_creativity"]].values[0][0]
            elif(df[(df["pupilId"] == pupilId)  & (df["taskId"] == taskId)][["subType"]].values == "multipleChoice"):
                if(df[(df["pupilId"] == pupilId)  & (df["taskId"] == taskId)][["falseAttempts"]].values == df[(df["pupilId"] == pupilId)  & (df["taskId"] == taskId)][["falseAttempts"]].values):
                    pupil_performance_df.loc[pupilId, taskId] = str(df[(df["pupilId"] == pupilId)  & (df["taskId"] == taskId)][["falseAttempts"]].values[0][0][0])
                elif(df[(df["pupilId"] == pupilId)  & (df["taskId"] == taskId)][["falseAttempts"]].values != df[(df["pupilId"] == pupilId)  & (df["taskId"] == taskId)][["falseAttempts"]].values):
                    pupil_performance_df.loc[pupilId, taskId] = str(df[(df["pupilId"] == pupilId)  & (df["taskId"] == taskId)][["state"]].values[0][0])
                #else:
                #    print("else")
                #    pupil_performance_df.loc[pupilId, taskId] = str([])
        elif((taskId not in df[df["pupilId"] == pupilId]["taskId"].unique()) & (task_df.loc[taskId, "subType"] == "multipleChoice")):
            #print("MC nicht gemacht")
            pupil_performance_df.loc[pupilId, taskId] = str([])
    
mcs = ["AI-2-v2-1-1", 'EvalML-1-4', 'AI-4-v2-1-1', 'AI-4-v2-1-2', 'AI-3-v2-1-2', 'EvalML-1-3', 'Classification-1-1-1', "Clustering-1-1-1", 'ML-3-1-2', "ML-3-1-3", "ML-3-1-4", 'EvalML-1-9', 'Classification-2-1-8', "Impact-1-1-1", 'EvalML-1-8', 'LimitsML-1-1']
for mcTaskId in mcs:
    pupil_performance_df[mcTaskId] = pupil_performance_df[mcTaskId].apply(ast.literal_eval)

for pupilId in pupil_performance_df.index:
    pupil_performance_df.loc[pupilId, "classId"] = df[df["pupilId"] == pupilId]["classId"].unique()[0]

pupil_performance_df.head()


,nr_tasks_per_pupil,total_duration in minutes,average duration per task in minutes,AI-1-v2-1-1,AI-4-v2-1-4,EvalML-1-1,ML-2-1-1,EvalML-1-5,AI-4-v2-1-5,EvalML-1-2,...,AIClosing-1-6,AIClosing-1-7,AIClosing-1-8,AIClosing-1-8_rating_content,AIClosing-1-8_rating_format,AIClosing-1-8_rating_creativity,AIClosing-1-8_correct_points,AIClosing-1-8_incorrect_points,AIClosing-1-8_irrelevant_points,classId
pupilId,,,,,,,,,,,,,,,,,,,,,
bniRycMi5bwrXjvGy,66,70.233333,1.064141,92.0,95.0,95.0,66.0,81.0,100.0,100.0,...,55.0,73.0,War alles gut.,5.0,5.0,5.0,NaN,NaN,NaN,qFEyvypMn4sAwLwtc
sqBXKssFvS8pFYAMv,66,66.650000,1.009848,89.0,99.0,96.0,79.0,93.0,100.0,98.0,...,74.0,89.0,Ich fand es gut das Videos eingebunden wurden ...,5.0,5.0,5.0,NaN,NaN,NaN,qFEyvypMn4sAwLwtc
Kd6FuRsrq6QvPrkKp,6,37.383333,6.230556,78.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,qFEyvypMn4sAwLwtc
6bBwnX2mwAp2NAwNR,35,114.550000,3.272857,77.0,68.0,NaN,52.0,NaN,100.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,qFEyvypMn4sAwLwtc
ZTvPk2i77LDC8pFmu,66,59.400000,0.900000,59.0,73.0,40.0,53.0,56.0,34.0,33.0,...,61.0,64.0,"Die Aufgaben waren zu schwer, und ab und zu ga...",5.0,5.0,5.0,NaN,NaN,NaN,qFEyvypMn4sAwLwtc


## Saving the DFs

In [26]:
pupil_performance_df.to_pickle('pupil_performance.pkl')
df.to_pickle('df.pkl')
chat_df.to_pickle("chat.pkl")
surveySlider.to_pickle('surveySlider.pkl')
inputFields.to_pickle('inputFields.pkl')
multipleChoice.to_pickle('multipleChoice.pkl')
freetext.to_pickle('freetext.pkl')
task_df.to_pickle('task_df.pkl')